## e-Commerce - Market Funnelling Analysis Project

Documenting the task

**Goal:** Analyze the customer journey from first impression to final purchase on an e-commerce website during October, with filters on price and on key product categories including electronics, appliances, furniture, and computers.

**Visualization:** Using bar graphs, histograms, and summary tables, the dashboard visualizes how users move through the marketing funnel and identifies where drop-offs occur to inform optimization strategies.

# Step 1. Preparing the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import gdown

# The gdown download failed previously, so we will now load the file from Google Drive.
# Path to your file in Google Drive
file_path = '/content/drive/MyDrive/e-Commerce-data-2019-Oct.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)


# Step 2. Analysing and Cleaning the Data

In [ ]:
# Basic information of the data

df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.8+ GB


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
# New functionality for category_code grouping

def extract_main_category(category_code):
    """
    Extracts the main category from a category_code string.
    If the category_code is NaN or None, it returns 'unknown'.
    """
    if pd.isna(category_code):
        return 'unknown'
    # Split by the first dot and take the first part
    return str(category_code).split('.')[0]

# Apply the function to create a new 'main_category' column
df['main_category'] = df['category_code'].apply(extract_main_category)

In [ ]:
# Since including unknown main category items into question will not be as
# useful as observing customer behavior with specific categories, it will be
# excluded in this investigation.

# Drop rows where 'category_code' and 'user_session'is null
df = df.dropna(subset=['category_code'])
df = df.dropna(subset=['user_session'])


In [ ]:
print("\nMissing Values after dropping rows with null category_code and user_session:")
display(df.isnull().sum())


Missing Values after dropping rows with null category_code and user_session:


,0
event_type,0
price,0
user_id,0
main_category,0


In [ ]:
columns_to_drop = ['event_time', 'product_id', 'category_id', 'category_code', 'brand', 'user_session']
# Check if columns exist before dropping to prevent KeyError on re-execution
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    df = df.drop(columns=existing_columns_to_drop)

print("\nUpdated dataframe after dropping:")
display(df.head())

print("\nSummary statistics after cleaning data")
display(df.describe())


Updated dataframe after dropping:


,event_type,price,user_id,main_category
1,view,33.20,554748717,appliances
2,view,543.10,519107250,furniture
3,view,251.74,550050854,computers
4,view,1081.98,535871217,electronics
5,view,908.62,512742880,computers



Summary statistics after cleaning data


,price,user_id
count,2.893315e+07,2.893315e+07
mean,3.395646e+02,5.337982e+08
std,3.755720e+02,1.856433e+07
min,0.000000e+00,3.386938e+07
25%,1.003600e+02,5.159537e+08
50%,2.007500e+02,5.301598e+08
75%,4.321200e+02,5.519934e+08
max,2.574070e+03,5.662807e+08


In [ ]:
# Reduce csv file to export to Excel file

sample_fraction = 1000000 / len(df)

# Take a random sample of the DataFrame
df_sampled = df.sample(frac=sample_fraction, random_state=42) # Using random_state for reproducibility

print(f"Original DataFrame rows: {len(df)}")
print(f"Sampled DataFrame rows: {len(df_sampled)}")

display(df_sampled.head())

# Export the sampled DataFrame to an Excel file
df_sampled.to_excel('e-Commerce-analysis.xlsx', index=False)
print("\nSampled DataFrame exported to e-Commerce-analysis.xlsx")

Original DataFrame rows: 28933153
Sampled DataFrame rows: 1000000


,event_type,price,user_id,main_category
39023283,view,424.21,553665323,computers
33807624,view,355.22,514748488,furniture
19415091,view,72.05,514901122,construction
26561547,view,43.73,518343373,appliances
15686744,view,82.34,534861499,appliances



Sampled DataFrame exported to e-Commerce-analysis.xlsx


# Step 3. Final Dashboard

In [1]:
#@title Tableau Dashboard
from IPython.display import HTML

HTML("""
<div class='tableauPlaceholder' id='viz1771389655544' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;e-&#47;e-CommerceCustomerAnalysis&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='e-CommerceCustomerAnalysis&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;e-&#47;e-CommerceCustomerAnalysis&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1771389655544');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1024px';vizElement.style.height='795px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1024px';vizElement.style.height='795px';} else { vizElement.style.width='100%';vizElement.style.height='1377px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
""")

#Final Assessment of e-Commerce Customer Behavior

[Link Tableau](https://public.tableau.com/views/e-CommerceCustomerAnalysis/Dashboard1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)

**Goal:** The goal of this project was to analyze customer behavior across the e-commerce marketing funnel to identify where potential customers drop off and uncover opportunities to improve conversion rates. Using a dataset of over 1 million event records, I built an interactive Tableau dashboard that tracks users through three key funnel stages: product view, add to cart, and purchase. The dashboard enables stakeholders to explore overall funnel performance and compare conversion rates across product categories, device types, and traffic sources, ultimately providing data-driven recommendations to increase sales and optimize marketing efforts.

**Conclusion:**

The analysis of 395,542 unique users revealed several key insights about customer behavior:

**Overall Funnel Performance**


*   4.39% of users who view a product eventually make a purchase
*   22.14% of viewers add an item to their cart


*   19.82% of users who add to cart complete their purchase

**The Biggest Drop-Off**

The most significant loss happens between View and Cart. Nearly 78% of users leave without adding any items. This suggests opportunities to improve product presentation, pricing clarity, or the add-to-cart experience.

**Category Performance**


*   Computers convert highest at approximately 6.5%

*   Home & Kitchen and Electronics also perform above average
*   Automotive and Kids categories convert below 3.5% , indicating potential issues with product selection or pricing

**Improvements and Suggestions:**

The dashboard could be enhanced by adding time trends to track conversion changes over weeks or months, and revenue metrics like average order value to measure business impact beyond conversion rates. Including tooltip annotations with exact drop-off percentages would improve clarity, while cohort analysis (new vs. returning users) could reveal whether marketing channels are attracting quality, loyal customers. These additions would provide deeper actionable insights for stakeholders.